In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

%matplotlib inline

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

dataPath = getDirectoryPathFromDialog('')

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def readStructureFactorValues(path, number_of_skipped_rows):
    return np.loadtxt(path, dtype=float, skiprows = number_of_skipped_rows)

def readStructureFactorValuesFromDialog(skiprows):
    return readStructureFactorValues(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def plotMultiplePotEnergySeries():
    Directory = getDirectoryPathFromDialog(dataPath);
    AllPotEnergySeries = glob.glob(Directory+"/PotEnergy*")
    NumberOfSeries = len(AllPotEnergySeries)
    Fig,Ax = plt.subplots(NumberOfSeries,2,squeeze=False, figsize=(17,5*NumberOfSeries))
    for i in range(NumberOfSeries):
        RunNumber = re.search(r'(?<=_)[\d]*(?=(.dat))',AllPotEnergySeries[i]).group()
        PotSeries = np.loadtxt(glob.glob(Directory+"/PotEnergy*_"+RunNumber+".dat")[0], dtype=float, skiprows = 1)
        PotEnergyAverage = np.mean(PotSeries)
        Ax[i,0].plot(PotSeries,'.')
        Ax[i,1].hist(PotSeries,bins = 20, density = True)
        Ax[i,1].vlines(PotEnergyAverage,0,0.004,color='g',label="PotEnergyAvg={}".format(round(PotEnergyAverage,2)))
        Ax[i,1].legend()
        print(RunNumber)

## Structure factor plotting

In [ ]:
StructureFactorResults = readStructureFactorValuesFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,1],'.',label=r'$S_{AA}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,2],'+',label=r'$S_{BB}$')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('strcture factor value')
Ax[0,0].legend()
Ax[0,0].set_title(r'$S_{AA}$'+' and '+r'$S_{BB}$' + ' as function of k')
Ax[0,0].set_xlim(0,1)
#Ax[0,0].set_ylim(0.71,0.74)
#Fig.savefig('SAA_SBB'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,3],'.')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('$S_{AB}(k)$')
Ax[0,0].set_title(r'$S_{AB}$')
Ax[0,0].set_xlim(0,1)
#Fig.savefig('SAB'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,4],'.')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('$S_{cc}(k)$')
Ax[0,0].set_title(r'$S_{cc}$')
Ax[0,0].set_xlim(0,1)
#Fig.savefig('Scc'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi = 200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,1],'-',label=r'$S_{AA}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,2],'-',label=r'$S_{BB}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,3],'-',label=r'$S_{AB}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,4],'-',label=r'$S_{cc}$')
Ax[0,0].legend()

## Ornstein-Zernike-fit

In [ ]:
def ModOZ(k,Amp,CorrL,Eta):
    return (Amp*((1.0+(k*CorrL)**2)**(-1.0+Eta*0.5)))

In [ ]:
LowerIndex = 7
UpperIndex = 50

In [ ]:
StructureFactorResults = readStructureFactorValuesFromDialog(1)

In [ ]:
popt, pcov = curve_fit(ModOZ,StructureFactorResults[LowerIndex:UpperIndex,0],StructureFactorResults[LowerIndex:UpperIndex,4],p0=[100,50,0.2],maxfev=50000,bounds=([0,0,0],[np.inf,np.inf,np.inf]))
print("Parameters found: Amp=", popt[0],", CorrL=", popt[1],", Eta=", popt[2])

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:UpperIndex,0], StructureFactorResults[:UpperIndex,4],'.')
Ax[0,0].plot(StructureFactorResults[LowerIndex:UpperIndex,0],ModOZ(StructureFactorResults[LowerIndex:UpperIndex,0],*popt))
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('$S_{cc}(k)$')
Ax[0,0].set_title(r'$S_{cc}$')
#Ax[0,0].set_xlim(0,1.5)
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
#Fig.savefig('Scc'+'.png',format='png')

## Positions plotting

In [ ]:
APositions, BPositions = readParticleConfigFromDialog(9)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(7,7),dpi =200)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b',ms=2)
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange',ms=2)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)
#Host[0,0].set_title('config for T=0.84')
#Fig.savefig('Particle_config_T=0.84_structure_factors_not_equilibrated'+'.png',format='png')

In [ ]:
plotMultiplePotEnergySeries()

## Comparison with Jürgen's computation

In [ ]:
StructureFactorsCorrectAA = readStructureFactorValuesFromDialog(0)

In [ ]:
StructureFactorsCorrectBB = readStructureFactorValuesFromDialog(0)

In [ ]:
StructureFactorsCorrectAB = readStructureFactorValuesFromDialog(0)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,1],'.',label=r'$S_{AA}$')
Ax[0,0].plot(StructureFactorsCorrectAA[:,0], StructureFactorsCorrectAA[:,1],'+',label=r'$S_{AA_{correct}}$',alpha=0.8)
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('strcture factor value')
Ax[0,0].legend()
#Ax[0,0].set_xlim(0,2.0)
#Ax[0,0].set_ylim(0,0.3)
#Fig.savefig('SAA_comparison_T=0.75_10_states'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,2],'.',label=r'$S_{BB}$')
Ax[0,0].plot(StructureFactorsCorrectBB[:,0], StructureFactorsCorrectBB[:,1],'+',label=r'$S_{BB_{correct}}$')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('strcture factor value')
Ax[0,0].legend()
#Ax[0,0].set_xlim(0,1)
#Ax[0,0].set_ylim(0.71,0.74)
#Fig.savefig('SAA_SBB'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,3],'.',label=r'$S_{AB}$')
Ax[0,0].plot(StructureFactorsCorrectAB[:,0], StructureFactorsCorrectAB[:,1],'+',label=r'$S_{AB_{correct}}$')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('strcture factor value')
Ax[0,0].legend()
#Ax[0,0].set_xlim(0,1)
#Ax[0,0].set_ylim(0.71,0.74)
#Fig.savefig('SAA_SBB'+'.png',format='png')